In [95]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import pandas as pd
import numpy as np
import json
from ssl import create_default_context
from datetime import datetime
from datetime import timedelta
#para request
import requests
from getpass import getpass
#parametros
import parametros

## Leyendo datos del indice ohmyfi-datosusuarios

In [96]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
)

## tomando fecha mas reciente del indice

In [97]:
total_docs = 0
try:
    response = es.search(
        index=parametros.ohmyfi_d_u_index,
        body={"aggs" : {
                   "max_date": {"max": {"field": "creado", "format": "yyyy-MM-dd HH:mm:ss"}}
                }
             },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["aggregations"]
    fecha_max=response["aggregations"]["max_date"]['value_as_string']
except:
    fecha_max = '2021-05-01 00:00:00'
print("Fecha maxima en indice usuarios: ",fecha_max)

Fecha maxima en indice usuarios:  2021-10-20 14:12:58


# Opcion creado

## Leyendo la APi ohmyfi-datosusuarios

In [98]:
now = datetime.now()
new_date = now - timedelta(days=6)
format = "%Y%m%d"
format_ES = "%Y.%m.%d"
fecha_url = str(new_date.strftime("%Y/%m/%d"))
fecha_hoy = str(now.strftime(format_ES))

In [99]:
h_fin = str(now.strftime("%Y/%m/%d %H:%M"))
h_inicio = str(( datetime.strptime(h_fin, '%Y/%m/%d %H:%M') - timedelta(hours=2)).strftime("%Y/%m/%d %H:%M"))

In [100]:
url = parametros.url_ohmyfi + '?/datosusuario' + '/=&apiKey=' + parametros.ohmyfi_api_key + '&opcion=creado' + '&inicio=' + str(h_inicio) + '&final=' + str(h_fin) 
#url = 'https://www.ohmyfi.com/ApiOMF?/datosusuario/=&apiKey=okMOpLAkiYpafQKXhXirwUys&opcion=creado&inicio=2021/05/28'

In [101]:
url

'https://www.ohmyfi.com/ApiOMF?/datosusuario/=&apiKey=okMOpLAkiYpafQKXhXirwUys&opcion=creado&inicio=2021/10/20 12:19&final=2021/10/20 14:19'

In [102]:
r = requests.get(url)
if r.status_code == 200:
    res = json.loads(r.text)
    datos_api = pd.DataFrame(res,columns=['lugar_cod','acepto_politicas','nombre','id_usuario','nombre_completo','apellidos'
                              ,'correo','documento','tipodoc','celular','telefono','direccion','ciudad'
                              ,'pais','creado','actualizado','eliminado','genero','fecha_nacimiento'
                              ,'nivel_educativo','rango_edad','alias','fecha','id_usuario_datos'
                              ,'validacion_datos','campos_validados','ocupacion','estado_civil','estrato'
                              ,'cantidad_hijos','indicativo_celular','nombre_corto'
                              ,'tipo_dispositivo','sistema_operativo'])
else:
    #exit()
    print(r.status_code)

In [104]:
datos_api

,lugar_cod,acepto_politicas,nombre,id_usuario,nombre_completo,apellidos,correo,documento,tipodoc,celular,...,validacion_datos,campos_validados,ocupacion,estado_civil,estrato,cantidad_hijos,indicativo_celular,nombre_corto,tipo_dispositivo,sistema_operativo
0,48624-ZZZY629,2021-10-20,VDA EL PLACER_HERVEO,00-08-22-1C-42-1F,Julio,None,None,1104674535,TI,None,...,None,None,None,None,None,None,,None,Smartphone,Android
1,21158-ZIFV597,2021-10-20,CL 10 10 11_SAN FRANCISCO,10-89-FB-58-72-B8,Brayan,Valencia Quintero,None,1037973367,TI,573104910351,...,None,None,None,None,None,None,+57,None,Smartphone,AndroidOS
2,49555-KPWM702,2021-10-20,VDA POTOSI_VILLAHERMOSA,EA-0F-40-D5-C2-19,Cristina,Díaz,None,1104705850,CC,None,...,None,None,None,None,None,None,,None,Smartphone,AndroidOS
3,26505-ZGYO204,2021-10-20,TUMBABARRETO_RIOSUCIO,FC-A6-21-EC-51-B6,Egleiidys,Arenas,None,24739906,CE,573148124608,...,None,None,None,None,None,None,+57,None,Smartphone,AndroidOS
4,30618-ZGYO428,2021-10-20,CUATRO VIENTOS_EL PASO,9E-02-B8-7E-1A-CA,Luis,Álvarez,None,1124002406,TI,573117436038,...,None,None,None,None,None,None,+57,None,Smartphone,AndroidOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,32010-ZZZY346,2021-10-20,SAN JOSE DE CANALETE_PUERTO ESCONDIDO,8E-94-9B-3E-72-0F,Olga Patricia,Bedoya Jaraba,olgabedoya17@hotmail.com,50909850,CC,573008010885,...,None,None,None,None,None,None,+57,None,Smartphone,AndroidOS
323,39350-ZIFV243,2021-10-20,EL VERGEL_SAN JUAN DE ARAMA,9C-BC-F0-56-92-34,Anyerli,Rojas Torres,anyik2312@gmail.com,1122118409,CC,573114967831,...,None,None,None,None,None,None,+57,None,Smartphone,AndroidOS
324,37504-VZVF913,2021-10-20,COSANZA_TIMANA,A0-41-47-F0-30-6A,Tania ximena,Silva oviedo,taniasilvaoviedo@gmail.com,1081517394,CC,573135665792,...,None,None,None,None,None,None,+57,None,Smartphone,AndroidOS
325,10786-ZGYO077,2021-10-20,VDA. CHONTALITO_SANTA BARBARA,04-D3-95-C4-72-A0,Juan Esteban,Soto,juanestebansotoescobar@gmail.com,1041350642,TI,573012455728,...,None,None,None,None,None,None,+57,None,Smartphone,AndroidOS


In [105]:
#datos_api.to_excel("crudov5.xlsx")

In [106]:


datos_api = datos_api.drop(datos_api[(datos_api['documento']=='')].index)

In [107]:
datos_api

,lugar_cod,acepto_politicas,nombre,id_usuario,nombre_completo,apellidos,correo,documento,tipodoc,celular,...,validacion_datos,campos_validados,ocupacion,estado_civil,estrato,cantidad_hijos,indicativo_celular,nombre_corto,tipo_dispositivo,sistema_operativo
0,48624-ZZZY629,2021-10-20,VDA EL PLACER_HERVEO,00-08-22-1C-42-1F,Julio,None,None,1104674535,TI,None,...,None,None,None,None,None,None,,None,Smartphone,Android
1,21158-ZIFV597,2021-10-20,CL 10 10 11_SAN FRANCISCO,10-89-FB-58-72-B8,Brayan,Valencia Quintero,None,1037973367,TI,573104910351,...,None,None,None,None,None,None,+57,None,Smartphone,AndroidOS
2,49555-KPWM702,2021-10-20,VDA POTOSI_VILLAHERMOSA,EA-0F-40-D5-C2-19,Cristina,Díaz,None,1104705850,CC,None,...,None,None,None,None,None,None,,None,Smartphone,AndroidOS
3,26505-ZGYO204,2021-10-20,TUMBABARRETO_RIOSUCIO,FC-A6-21-EC-51-B6,Egleiidys,Arenas,None,24739906,CE,573148124608,...,None,None,None,None,None,None,+57,None,Smartphone,AndroidOS
4,30618-ZGYO428,2021-10-20,CUATRO VIENTOS_EL PASO,9E-02-B8-7E-1A-CA,Luis,Álvarez,None,1124002406,TI,573117436038,...,None,None,None,None,None,None,+57,None,Smartphone,AndroidOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,32010-ZZZY346,2021-10-20,SAN JOSE DE CANALETE_PUERTO ESCONDIDO,8E-94-9B-3E-72-0F,Olga Patricia,Bedoya Jaraba,olgabedoya17@hotmail.com,50909850,CC,573008010885,...,None,None,None,None,None,None,+57,None,Smartphone,AndroidOS
323,39350-ZIFV243,2021-10-20,EL VERGEL_SAN JUAN DE ARAMA,9C-BC-F0-56-92-34,Anyerli,Rojas Torres,anyik2312@gmail.com,1122118409,CC,573114967831,...,None,None,None,None,None,None,+57,None,Smartphone,AndroidOS
324,37504-VZVF913,2021-10-20,COSANZA_TIMANA,A0-41-47-F0-30-6A,Tania ximena,Silva oviedo,taniasilvaoviedo@gmail.com,1081517394,CC,573135665792,...,None,None,None,None,None,None,+57,None,Smartphone,AndroidOS
325,10786-ZGYO077,2021-10-20,VDA. CHONTALITO_SANTA BARBARA,04-D3-95-C4-72-A0,Juan Esteban,Soto,juanestebansotoescobar@gmail.com,1041350642,TI,573012455728,...,None,None,None,None,None,None,+57,None,Smartphone,AndroidOS


In [108]:
#datos_api.to_excel("crudov5.xlsx") si se borran por falta de documento se imprime el excel para validar

In [109]:
datos_api['nombre'] = datos_api['nombre'].str.replace('"','')
datos_api['creado'] = datos_api['creado'].str.replace('"','')
datos_api['actualizado'] = datos_api['actualizado'].str.replace('"','')
datos_api['eliminado'] = datos_api['eliminado'].str.replace('"','')

### Ajustando formato de fechas

In [110]:
try:
    datos_api['creado'] = datos_api["creado"].str.split(".", n = 1, expand = True)[0]
    datos_api['actualizado'] = datos_api["actualizado"].str.split(".", n = 1, expand = True)[0]
    datos_api['eliminado'] = datos_api["eliminado"].str.split(".", n = 1, expand = True)[0]
except:
    pass

In [111]:
#datos_api.to_excel("antes_de_datosqueestan.xlsx")

### Descartando datos de la API que ya están en el indice

In [112]:
datos_api = datos_api.drop(datos_api[(datos_api['creado']<= fecha_max)].index)

In [113]:
datos_api

,lugar_cod,acepto_politicas,nombre,id_usuario,nombre_completo,apellidos,correo,documento,tipodoc,celular,...,validacion_datos,campos_validados,ocupacion,estado_civil,estrato,cantidad_hijos,indicativo_celular,nombre_corto,tipo_dispositivo,sistema_operativo
311,48164-ZGYO701,2021-10-20,SAN JOSE DE LAS HERMOSAS_CHAPARRAL,FC-D9-08-D4-06-80,Andrés,Tapiero,None,1193502400,CC,None,...,None,None,None,None,None,None,,None,Smartphone,AndroidOS
312,18953-ZGYO164,2021-10-20,BUENOS AIRES_ARBOLETES,72-44-4B-1A-5A-F6,Juan,Gonzalez,None,123456789,TI,None,...,None,None,None,None,None,None,,None,Smartphone,AndroidOS
313,18842-ZIFV808,2021-10-20,SANTA FÉ DE ANTIOQUIA_LAURELES,08-AA-55-9E-C2-A2,Sor María,Zapata Zapata,None,1022092767,CC,573026083033,...,None,None,None,None,None,None,+57,None,Smartphone,AndroidOS
314,37583-ZGYO493,2021-10-20,CGTO CARRETALITO_BARRACAS,88-B2-91-34-D3-76,Liz,Carrillo,None,1193082308,CC,None,...,None,None,None,None,None,None,,None,Smartphone,iOS
315,30804-KPWM989,2021-10-20,CORREG. EL MARQUEZ_RIO DE ORO,28-E3-47-A3-CC-8C,RULDIS PAOLA,URQUIJO CARRASCAL,ruldis09@hotmail.com,1065875376,CC,573146387814,...,None,None,None,None,None,None,+57,None,PC,Windows
316,74388-ZZZY039,2021-10-20,SAN PASCUAL_CANASGORDAS,38-80-DF-0B-07-A0,Usua,Rodríguez,isa@gamail.com,440067942,CE,573155876662,...,None,None,None,None,None,None,+57,None,Smartphone,Android
317,21891-ZGYO173,2021-10-20,CORREG. EL DOS_TURBO,04-E5-98-6B-9A-57,Edwin,None,None,1028027980,CC,None,...,None,None,None,None,None,None,,None,Smartphone,AndroidOS
318,31558-ZGYO327,2021-10-20,VEREDA GUAYABAL_LORICA,F2-D1-60-21-F6-E0,Eliana,Florez,None,1063152746,CC,573233512341,...,None,None,None,None,None,None,+57,None,Smartphone,AndroidOS
319,38738-ZGYO658,2021-10-20,VDA EL CABLE_EL CASTILLO,48-3F-E9-F6-37-68,Elizabeth,Urrego,isabelurrego78@gmail.com,1121419634,CC,573103436076,...,None,None,None,None,None,None,+57,None,Smartphone,AndroidOS
320,48305-ZZZY024,2021-10-20,BUENAVISTA_COYAIMA,8C-7A-3D-E3-F1-F8,Clemente,Dajome,None,1030684015,CC,None,...,None,None,None,None,None,None,,None,Smartphone,AndroidOS


In [114]:
datos_api.to_excel("qeyaestan_datos_usuarios.xlsx")

In [115]:
indice = 'ohmyfi-datopruebaingesta1'

In [116]:
use_these_keys = ['lugar_cod','acepto_politicas','nombre','id_usuario','nombre_completo','apellidos'
                              ,'correo','documento','tipodoc','celular','telefono','direccion','ciudad'
                              ,'pais','creado','actualizado','eliminado','genero','fecha_nacimiento'
                              ,'nivel_educativo','rango_edad','alias','fecha','id_usuario_datos'
                              ,'validacion_datos','campos_validados','ocupacion','estado_civil','estrato'
                              ,'cantidad_hijos','indicativo_celular','nombre_corto'
                              ,'tipo_dispositivo','sistema_operativo','@timestamp']
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

timestamp = datetime.now()
datos_api['@timestamp'] = timestamp.isoformat()
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{document['tipodoc']+document['documento']}",
                "_source": filterKeys(document),
            }
salida = helpers.bulk(es, doc_generator(datos_api))
print("Fecha: ", now, "- Documentos nuevos: ", salida[0])

Fecha:  2021-10-20 14:19:44.594817 - Documentos nuevos:  16


# Opcion actualizado

In [117]:
#url = 'https://www.ohmyfi.com/ApiOMF?/datosusuario/=&apiKey=okMOpLAkiYpafQKXhXirwUys&opcion=actualizado&inicio='+ fecha_url
#url = parametros.url_ohmyfi + '?/datosusuario' + '/=&apiKey=' + parametros.ohmyfi_api_key + '&opcion=actualizado&inicio=' + fecha_url

#modifica_para prueba____url = parametros.url_ohmyfi + '?/datosusuario' + '/=&apiKey=' + parametros.ohmyfi_api_key + '&opcion=actualizado' + '&inicio=' + str(h_inicio) + '&final=' + str(h_fin)

url = 'https://www.ohmyfi.com/ApiOMF?/datosusuario/=&apiKey=okMOpLAkiYpafQKXhXirwUys&opcion=actualizado&inicio=2021/10/20 13:14&final=2021/10/20 13:23'
r = requests.get(url)
r = requests.get(url)
if r.status_code == 200:
    res = json.loads(r.text)
    datos_api_act  = pd.DataFrame(res,columns=['lugar_cod','acepto_politicas','nombre','id_usuario','nombre_completo','apellidos'
                              ,'correo','documento','tipodoc','celular','telefono','direccion','ciudad'
                              ,'pais','creado','actualizado','eliminado','genero','fecha_nacimiento'
                              ,'nivel_educativo','rango_edad','alias','fecha','id_usuario_datos'
                              ,'validacion_datos','campos_validados','ocupacion','estado_civil','estrato'
                              ,'cantidad_hijos','indicativo_celular','nombre_corto'
                              ,'tipo_dispositivo','sistema_operativo'])
else:
    exit()

In [118]:
url

'https://www.ohmyfi.com/ApiOMF?/datosusuario/=&apiKey=okMOpLAkiYpafQKXhXirwUys&opcion=actualizado&inicio=2021/10/20 13:14&final=2021/10/20 13:23'

In [119]:
datos_api_act

,lugar_cod,acepto_politicas,nombre,id_usuario,nombre_completo,apellidos,correo,documento,tipodoc,celular,...,validacion_datos,campos_validados,ocupacion,estado_civil,estrato,cantidad_hijos,indicativo_celular,nombre_corto,tipo_dispositivo,sistema_operativo
0,14715-ZGYO346,2021-10-20,PAPAYAL_BARRANCAS,78-D7-5F-6E-F2-2D,Naiyelis,Urdaneta,gosaleznayelis@gmail.com,1118878474,CC,573004053442,...,None,None,None,None,None,None,+57,None,Smartphone,iOS
1,19879-ZZZY074,2021-10-20,BELLAVISTA_DONMATIAS,C4-2A-D0-26-58-9E,Natalia,Cataño,catanocatanonatalia@gmail.com,1007373248,CC,573135133634,...,None,None,None,None,None,None,+57,None,Smartphone,iOS
2,26802-ZIFV833,2021-10-20,VEREDA HOJAS ANCHAS_SUPIA,48-D8-90-A9-51-61,juan sebastian,vasquez tobon,juansebastianvasqueztobon@gmail.com,1038626153,TI,573192832180,...,None,None,None,None,None,None,+57,None,PC,Windows
3,30220-ZZZY110,2021-10-20,BARRANCALEBRIJA_AGUACHICA,FA-3D-63-0E-4F-1B,Jesu,Otero Torrecilla,costajot@gmail.com,1030589365,CC,573178690699,...,None,None,None,None,None,None,+57,None,Smartphone,AndroidOS
4,31648-ZGYO908,2021-10-20,VDA SITIO NUEVO_LORICA,D8-0B-9A-92-59-8C,Marlene,Arteaga argel,None,30655074,CC,573106263324,...,None,None,None,None,None,None,+57,None,Smartphone,AndroidOS
5,48538-ZZZY772,2021-07-03,VDA LOS ANDES_FRESNO,D0-05-E4-BB-A0-72,Deicy,Pérez,demarcelaperez@gmail.com,1109296871,CC,573204876442,...,None,None,None,None,None,None,+57,None,Smartphone,Android
6,47510-ZZZY594,2021-10-20,VILLA LOPEZ_SAN JUAN DE BETULIA,EA-C3-D5-DD-28-4F,Adriana,Osorio,adryog95@gmail.com,1005647115,CC,573217385981,...,None,None,None,None,None,None,+57,None,Smartphone,AndroidOS
7,39161-ZZZY478,2021-10-20,SAN MIGUEL_PUERTO GAITAN,AA-CA-9D-2D-7D-B6,Ana matilde,Niño moreno,ananino492@gmail.com,3132383255,CC,573132383255,...,None,None,None,None,None,None,+57,None,Smartphone,AndroidOS
8,22432-ZIFV600,2021-10-20,CORREG. BUENOS AIRES-PALIZADAS_ZARAGOZA,6E-C6-59-09-D9-5F,Cristián,Diaz,cristiandiaz471@gmail.con,3239536595,TI,573234217344,...,None,None,None,None,None,None,+57,None,Smartphone,AndroidOS
9,73919-ZZZY238,2021-07-26,RESGUARDO INDIGENA DE CAICEMAPA_DISTRACCION,A0-41-47-49-D4-23,Jose Julio,Galindo Chinchia,josegalindo2031@gmail.com,1192789282,CC,573816665002,...,None,None,None,None,None,None,+57,None,Smartphone,Android


In [120]:
datos_api_act.to_excel("actu81021_datos_usuarios.xlsx")

In [121]:
datos_api_act.fillna('', inplace=True)
datos_api_act = datos_api_act.drop(datos_api_act[(datos_api_act['documento']=='')].index)
datos_api_act['nombre'] = datos_api_act['nombre'].str.replace('"','')
datos_api_act['creado'] = datos_api_act['creado'].str.replace('"','')
datos_api_act['actualizado'] = datos_api_act['actualizado'].str.replace('"','')
datos_api_act['eliminado'] = datos_api_act['eliminado'].str.replace('"','')

In [122]:
datos_api_act

,lugar_cod,acepto_politicas,nombre,id_usuario,nombre_completo,apellidos,correo,documento,tipodoc,celular,...,validacion_datos,campos_validados,ocupacion,estado_civil,estrato,cantidad_hijos,indicativo_celular,nombre_corto,tipo_dispositivo,sistema_operativo
0,14715-ZGYO346,2021-10-20,PAPAYAL_BARRANCAS,78-D7-5F-6E-F2-2D,Naiyelis,Urdaneta,gosaleznayelis@gmail.com,1118878474,CC,573004053442,...,,,,,,,+57,,Smartphone,iOS
1,19879-ZZZY074,2021-10-20,BELLAVISTA_DONMATIAS,C4-2A-D0-26-58-9E,Natalia,Cataño,catanocatanonatalia@gmail.com,1007373248,CC,573135133634,...,,,,,,,+57,,Smartphone,iOS
2,26802-ZIFV833,2021-10-20,VEREDA HOJAS ANCHAS_SUPIA,48-D8-90-A9-51-61,juan sebastian,vasquez tobon,juansebastianvasqueztobon@gmail.com,1038626153,TI,573192832180,...,,,,,,,+57,,PC,Windows
3,30220-ZZZY110,2021-10-20,BARRANCALEBRIJA_AGUACHICA,FA-3D-63-0E-4F-1B,Jesu,Otero Torrecilla,costajot@gmail.com,1030589365,CC,573178690699,...,,,,,,,+57,,Smartphone,AndroidOS
4,31648-ZGYO908,2021-10-20,VDA SITIO NUEVO_LORICA,D8-0B-9A-92-59-8C,Marlene,Arteaga argel,,30655074,CC,573106263324,...,,,,,,,+57,,Smartphone,AndroidOS
5,48538-ZZZY772,2021-07-03,VDA LOS ANDES_FRESNO,D0-05-E4-BB-A0-72,Deicy,Pérez,demarcelaperez@gmail.com,1109296871,CC,573204876442,...,,,,,,,+57,,Smartphone,Android
6,47510-ZZZY594,2021-10-20,VILLA LOPEZ_SAN JUAN DE BETULIA,EA-C3-D5-DD-28-4F,Adriana,Osorio,adryog95@gmail.com,1005647115,CC,573217385981,...,,,,,,,+57,,Smartphone,AndroidOS
7,39161-ZZZY478,2021-10-20,SAN MIGUEL_PUERTO GAITAN,AA-CA-9D-2D-7D-B6,Ana matilde,Niño moreno,ananino492@gmail.com,3132383255,CC,573132383255,...,,,,,,,+57,,Smartphone,AndroidOS
8,22432-ZIFV600,2021-10-20,CORREG. BUENOS AIRES-PALIZADAS_ZARAGOZA,6E-C6-59-09-D9-5F,Cristián,Diaz,cristiandiaz471@gmail.con,3239536595,TI,573234217344,...,,,,,,,+57,,Smartphone,AndroidOS
9,73919-ZZZY238,2021-07-26,RESGUARDO INDIGENA DE CAICEMAPA_DISTRACCION,A0-41-47-49-D4-23,Jose Julio,Galindo Chinchia,josegalindo2031@gmail.com,1192789282,CC,573816665002,...,,,,,,,+57,,Smartphone,Android


### Ajustando formato de fechas

In [123]:
try:
    datos_api_act['creado'] = datos_api_act["creado"].str.split(".", n = 1, expand = True)[0]
    datos_api_act['actualizado'] = datos_api_act["actualizado"].str.split(".", n = 1, expand = True)[0]
    datos_api_act['eliminado'] = datos_api_act["eliminado"].str.split(".", n = 1, expand = True)[0]
except:
    pass

### Descartando datos de la API que ya están en el indice

In [124]:
datos_api_act = datos_api_act.drop(datos_api_act[(datos_api_act['actualizado']<= fecha_max)].index)

In [125]:
datos_api_act

,lugar_cod,acepto_politicas,nombre,id_usuario,nombre_completo,apellidos,correo,documento,tipodoc,celular,...,validacion_datos,campos_validados,ocupacion,estado_civil,estrato,cantidad_hijos,indicativo_celular,nombre_corto,tipo_dispositivo,sistema_operativo


In [126]:
datos_api_act.to_excel("actu3_datos_usuarios_cruce.xlsx")

### Escribiendo actualizados al indice

In [127]:
timestamp = datetime.now()
datos_api_act['@timestamp'] = timestamp.isoformat()
salida = helpers.bulk(es, doc_generator(datos_api_act))
print("Fecha: ", now, "- Documentos actualizados: ", salida[0])

Fecha:  2021-10-20 14:19:44.594817 - Documentos actualizados:  0
